### This is my work in progress exploring this dataset. 
#### Brian of London

In [140]:
import pandas as pd
import csv
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio

In [141]:
df = pd.read_csv('data/nyt_ts.csv', index_col='timestamp')
states = df['state'].unique()

In [142]:
df['votes_prev'] = df.groupby('state')['votes'].shift(1)
df['votes_diff'] = [v - vp for v,vp in zip(df['votes'],df['votes_prev'])]
df.fillna(0, inplace=True)


In [143]:
# Calculate absolute vote values per time stamp
df['trump_v'] = [round(sh * v) for sh,v in zip(df['vote_share_rep'],df['votes_diff'])]
df['biden_v'] = [round(sh * v) for sh,v in zip(df['vote_share_dem'],df['votes_diff'])]
df['trd_v']= [round(sh * v) for sh,v in zip(df['vote_share_trd'],df['votes_diff'])]
df['check_v'] = [d-(a+b+c) for a,b,c,d in zip(df['trump_v'],df['biden_v'],df['trd_v'],df['votes_diff'])]

df['trump_c'] = [round(sh * v) for sh,v in zip(df['vote_share_rep'],df['votes'])]
df['biden_c'] = [round(sh * v) for sh,v in zip(df['vote_share_dem'],df['votes'])]
df['trd_c']= [round(sh * v) for sh,v in zip(df['vote_share_trd'],df['votes'])]
df['check_c'] = [d-(a+b+c) for a,b,c,d in zip(df['trump_c'],df['biden_c'],df['trd_c'],df['votes'])]

# check_v is showing rounding errors
df.groupby('state').last()

,Unnamed: 0,votes,eevp,eevp_source,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,vote_share_dem,...,votes_prev,votes_diff,trump_v,biden_v,trd_v,check_v,trump_c,biden_c,trd_c,check_c
state,,,,,,,,,,,,,,,,,,,,,
alabama,179,2309900,99,edison,2300000,1318255,2074338,2123372,0.621,0.365,...,2297216.0,12684.0,7877,4630,0,177.0,1434448,843114,0,32338
alaska,5,240234,75,edison,367000,163387,300495,318608,0.580,0.381,...,190970.0,49264.0,28573,18770,0,1921.0,139336,91529,0,9369
arizona,429,3348758,98,edison,3400000,1252401,2306559,2604657,0.490,0.494,...,3343467.0,5291.0,2593,2614,0,84.0,1640891,1654286,0,53581
arkansas,319,1209372,99,edison,1225000,684872,1069468,1130635,0.626,0.346,...,1209360.0,12.0,8,4,0,0.0,757067,418443,0,33862
california,600,15975229,95,edison,16800000,4483810,13038547,14237884,0.336,0.643,...,15937716.0,37513.0,12604,24121,0,788.0,5367677,10272072,0,335480
colorado,716,3212169,97,edison,3300000,1202484,2569520,2780247,0.419,0.554,...,3173957.0,38212.0,16011,21169,0,1032.0,1345899,1779542,0,86728
connecticut,941,1821878,98,edison,1850000,673215,1558993,1644920,0.392,0.592,...,1821824.0,54.0,21,32,0,1.0,714176,1078552,0,29150
delaware,952,504010,99,edison,510000,185127,413921,443814,0.398,0.588,...,502392.0,1618.0,644,951,0,23.0,200596,296358,0,7056
florida,1237,11075446,99,edison,11100000,4617886,8474179,9501617,0.512,0.478,...,11075806.0,-360.0,-184,-172,0,-4.0,5670628,5294063,0,110755


In [144]:
# Need to find the growing increments of votes for each time by each state
filt = (df['state'] == 'alaska')
dfs = df[filt]
dfs

,Unnamed: 0,votes,eevp,eevp_source,state,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,...,votes_prev,votes_diff,trump_v,biden_v,trd_v,check_v,trump_c,biden_c,trd_c,check_c
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-11-04T10:01:44Z,0,0,0,edison,alaska,367000,163387,300495,318608,0.000,...,0.0,0.0,0,0,0,0.0,0,0,0,0
2020-11-04T07:47:41Z,1,113744,31,edison,alaska,367000,163387,300495,318608,0.605,...,0.0,113744.0,68815,40038,0,4891.0,68815,40038,0,4891
2020-11-04T08:57:55Z,2,132475,36,edison,alaska,367000,163387,300495,318608,0.614,...,113744.0,18731.0,11501,6500,0,730.0,81340,45969,0,5166
2020-11-04T18:34:58Z,3,172031,56,edison,alaska,367000,163387,300495,318608,0.629,...,132475.0,39556.0,24881,13053,0,1622.0,108207,56770,0,7054
2020-11-09T19:09:13Z,4,190970,61,edison,alaska,367000,163387,300495,318608,0.622,...,172031.0,18939.0,11780,6364,0,795.0,118783,64166,0,8021
2020-11-11T02:46:15Z,5,240234,75,edison,alaska,367000,163387,300495,318608,0.580,...,190970.0,49264.0,28573,18770,0,1921.0,139336,91529,0,9369


In [ ]:
fig = px.scatter(df,x=dfs.index,y=dfs['votes'].diff())
fig.show()

In [ ]:
states = df['state'].unique()
filt = (df['state'] == 'illinois')
df[filt]

In [ ]:
fig = px.scatter_3d(df, x=df.index, y=df['state'], z=df['trump_v'],color=df['state'])
fig.show()
pio.write_html(fig, file=f'3d.html', auto_open=True)
